In [1]:
from dataclasses import dataclass, field
from collections.abc import Callable, Mapping, Sequence
import itertools
import numpy as np
from scipy.optimize import minimize
import xarray as xr

from HARK import distribution

In [2]:
from stage import Stage

# General Bellman Stage Form

The agent:
 - begins in some input states $\vec{X} \in \vec{X}$
 - experiences some exogeneous shocks $\vec{k} \in \vec{K}$
 - can choose some actions $\vec{a} \in \vec{A}$
 - subject to constraints $\Gamma: \vec{X} \times \vec{K} \times \vec{A} \rightarrow \mathbb{B}$
 - experience a reward $F: \vec{X} \times \vec{K} \times \vec{A} \rightarrow \mathbb{R}$
 - together, these determine some output states $\vec{y} \in \vec{Y}$ via...
 - a **deterministic** transition function $T: \vec{X} \times \vec{K} \times \vec{A} \rightarrow \vec{Y}$
   - _This is deterministic because shocks have been isolated to the beginning of the stage._
   - CDC thinks there needs to be an additional between-stage transition function.
 - The agent has a discount factor $\beta$ for future utility.
   - CDC: These can be stochastic!

## Solving one stage

For any stage, consider two value functions.
 - $v_x : X \rightarrow \mathbb{R}$ is the value of its input states
 - $v_y : Y \rightarrow \mathbb{R}$ is the value of its output states. Others migth write this $\mathfrak{v}$'
 - $\hat{v}_y : X \times K \times A \rightarrow \mathbb{R} $
 
The stage is solved with respect to a value function $v_y : \vec{Y} \rightarrow \mathbb{R}$ over the output states. The $q: \vec{X} \times \vec{K} \times \vec{A} \rightarrow \mathbb{R}$ is the value of a state, shock, action combination.

$$q(\vec{x}, \vec{k}, \vec{a}) = F(\vec{x}, \vec{k}, \vec{a}) + \overbrace{\beta(\vec{x},\vec{k},\vec{a}) v_y(T(\vec{x}, \vec{k}, \vec{a}))}^{\hat{v}_y}$$

where $\beta$ is the agent's discount factor for that stage. Note that there is no expecation taking in this operation because $T$ is deterministic.

The optimal policy $\pi: \vec{X} \times \vec{K} \rightarrow \vec{A}$ is:

$$\pi^*(\vec{x}, \vec{k}) = \mathrm{argmax}_{\vec{a} \in \vec{A}} q(\vec{x}, \vec{k}, \vec{a})$$

(This is solved by griding over $x$ and $k$ ...)

The optimal policy $\pi^*$ can then be used to derive the value function over the input states $V_x: \vec{X} \rightarrow \mathbb{R}$.

$$v_x(\vec{x}) = \mathbb{E}_{\vec{k} \in \vec{K}}[q(\vec{x}, \vec{k}, \pi^*(\vec{x}, \vec{k}))]$$

Note that this requires no optimization, but does require the taking of expectations over the probability distribution over the shocks.

**TODO**: Infinite horizon portfolio choice solver with arbitrary grids

**TODO**: Forward simulation

**TODO** : Shocks
 - Forward-simulation with discretized shock values
 
**TODO**: Parameters
 - Allowing for parameters in stage function scope (e.g., CRRA, PermGroFac)
 
**TODO**: Fix $\hat{v}_y$ with $\beta{x, k, a}$
 
**TODO**: Type aliases for the function domains $X$, $A$, $K$, etc.

# Portfolio Choice Problem

## Stages

First we define the stages. Then we combine the stages together and solve them recursively.

### Consumption stage

Example: The consumption stage:

* $c \in A_0 = \mathbb{R}$
* $m \in X_0 = \mathbb{R}$
* $a \in Y_0 = \mathbb{R}$
* $\Gamma_0$ ... restricts consumption $c \leq m$
* $F_0(m,c) = CRRA(c, \rho)$
* $T_0(m,c) = m - c$ 
* $\beta_0 = \beta $

Requires a parameter $\rho$

In [3]:
from HARK.utilities import CRRAutility

CRRA = 5

consumption_stage = Stage(
    transition = lambda x, k, a : {'a' : x['m'] - a['c']}, 
    reward = lambda x, k, a : CRRAutility(a['c'], CRRA), 
    inputs = ['m'], 
    actions = ['c'],
    outputs = ['a'],
    constraints = [lambda x, k, a: x['m'] - a['c']], # has to be nonnegative to clear
    discount = .96
)

In [4]:
def consumption_v_y(y : Mapping[str,...]):
    return CRRAutility(y['a'], CRRA)

pi_star, q = consumption_stage.optimal_policy({'m' : [9, 11, 20, 300, 4000, 5500]}, v_y = consumption_v_y)

q

/home/sb/projects/HARK/HARK/utilities.py:141: RuntimeWarning: divide by zero encountered in double_scalars
  return c ** (1.0 - gam) / (1.0 - gam)


<xarray.DataArray (m: 6)>
array([-1.19474070e-03, -5.35393332e-04, -4.89918359e-05, -9.67739967e-10,
       -3.06198974e-14, -8.56629327e-15])
Coordinates:
  * m        (m) int64 9 11 20 300 4000 5500

In [5]:
consumption_stage.T({'m' : 100}, {}, {'c' : 50})

{'a': 50}

In [6]:
consumption_stage.T({'m' : 100}, {}, {'c' : 101})

{'a': -1}

In [7]:
consumption_stage.reward({'m' : 100}, {}, {'c' : 50})

-4e-08

In [8]:

consumption_stage.q({'m' : 100}, {}, {'c' : 50}, v_y = consumption_v_y)

-7.84e-08

In [9]:
consumption_stage.v_x_expectations({'m' : [0, 50, 100, 1000]}, {}, consumption_v_y)

<xarray.DataArray (m: 4)>
array([-1.29075010e+17, -1.25419100e-06, -7.83869374e-08, -7.83869374e-12])
Coordinates:
  * m        (m) int64 0 50 100 1000

### Allocation stage

The allocation stage. Note that this is a trivial transition function.:

* $\alpha \in A_1 = \mathbb{R}$
* $a \in X_1 = \mathbb{R}$
* $(a, \alpha) \in Y_1 = \mathbb{R}^2$
* $\Gamma_1$ ... restricts allocation $0 \leq \alpha \leq 1$
* $F_1(a,\alpha) = 0$
* $T_1(a,\alpha) = (a, \alpha)$
* $\beta_1 = 1 $

In [10]:
allocation_stage = Stage(
    transition = lambda x, k, a : {'a' : x['a'], 'alpha' : a['alpha']}, 
    inputs = ['a'], 
    actions = ['alpha'],
    outputs = ['a', 'alpha'],
    constraints = [
        lambda x, k, a: 1 - a['alpha'], 
        lambda x, k, a: a['alpha']
    ]
)

In [11]:
allocation_stage.T({'a': 100}, {}, {'alpha' : 0.5})

{'a': 100, 'alpha': 0.5}

In [12]:
allocation_stage.T({'a': 100}, {}, {'alpha' : -0.1})

{'a': 100, 'alpha': -0.1}

In [13]:
allocation_stage.reward({'a': 100}, {}, {'alpha' : 0.5})

0

Optimize portfolio allocation $\alpha$ with a more complex value function:

In [14]:
def allocation_v_y(y : Mapping[str,...]):
    return CRRAutility(y['alpha'] * y['a'] + 1,CRRA) \
            + CRRAutility((1 - y['alpha']) * y['a'] + 1, CRRA * 0.9) 

pi_star, q = allocation_stage.optimal_policy({'a' : [9, 11, 20, 300, 4000, 5500]}, v_y = consumption_v_y)

q

<xarray.DataArray (a: 6)>
array([-3.81039476e-05, -1.70753364e-05, -1.56250000e-06, -3.08641975e-11,
       -9.76562500e-16, -2.73205382e-16])
Coordinates:
  * a        (a) int64 9 11 20 300 4000 5500

In [15]:
allocation_stage.v_x_expectations({'a' : [0, 50, 100, 1000]}, {}, allocation_v_y)

<xarray.DataArray (a: 4)>
array([-5.35714286e-01, -2.99296597e-06, -2.49530715e-07, -5.69496124e-11])
Coordinates:
  * a        (a) int64 0 50 100 1000

### Growth stage

The growth stage stage:

* $A_2 = \emptyset$
* $(a, \alpha) \in X_2 = \mathbb{R}^2$
* $m \in Y_0 = \mathbb{R}$
* Shocks:
    * $\psi \sim \text{Lognormal}(0,\sigma_\psi)$
    * $\theta \sim \text{Lognormal}(0,\sigma_\theta)$
    * $\eta \sim \text{Lognormal}(0,\sigma_\eta)$
* $F_2(a,\alpha) = 0$
* $T_2(a,\alpha) =  \frac{(\alpha \eta + (1 - \alpha) R) a + \theta}{\psi G} $ 

Requires parameters $R$ and $G$

In [16]:
R = 1.01
G = 1.02

sigma_psi = 1.05
sigma_theta = 1.15
sigma_eta = 1.1
p_live = 0.98

def growth_transition(x, k, a): 
    return {'m' : ((x['alpha'] * k['eta'] + (1 - x['alpha']) * R) 
                   * x['a'] + k['theta']) 
            / (k['psi'] * G)}

growth_stage = Stage(
    transition = growth_transition,
    inputs = ['a', 'alpha'],
    discount = p_live, ## Later, replace this with a shock!
    shocks = {
        'psi' : distribution.Lognormal(0, sigma_psi),
        'theta' : distribution.Lognormal(0, sigma_theta),
        'eta' : distribution.Lognormal(0, sigma_eta),
        # 'live' : distribution.Bernoulli(p_live) ## Not implemented for now
    },
    outputs = ['m'],
)

In [17]:
growth_stage.T(
    {'a': 100, 'alpha' : 0.5},
    {'psi' : 1.00, 'theta' : 1.10, 'eta' : 1.05, 'live' : 1},
    {}
)

{'m': 102.05882352941175}

**TODO**: What is this returning? There ar no actions to optimize!

In [18]:
def growth_v_y(y : Mapping[str,...]):
    return CRRAutility(y['m'], CRRA) # * 'live' ?

pi_star, q = growth_stage.optimal_policy(
    {'a' : [300, 600],
     'alpha' : [0, 1.0]
    },
    {'psi' : [1., 1.1], 
     'theta' : [1., 1.1], 
     'eta' : [1., 1.1],
     # 'live' : [0, 1] 
    }, 
    v_y = growth_v_y)

q

<xarray.DataArray (a: 2, alpha: 2, psi: 2, theta: 2, eta: 2)>
array([[[[[-3.10507745e-11, -3.10507745e-11],
          [-3.10099518e-11, -3.10099518e-11]],

         [[-4.54614389e-11, -4.54614389e-11],
          [-4.54016704e-11, -4.54016704e-11]]],


        [[[-3.23073093e-11, -2.20930053e-11],
          [-3.22644116e-11, -2.20663269e-11]],

         [[-4.73011315e-11, -3.23463690e-11],
          [-4.72383250e-11, -3.23073093e-11]]]],



       [[[[-1.95349366e-12, -1.95349366e-12],
          [-1.95220688e-12, -1.95220688e-12]],

         [[-2.86011007e-12, -2.86011007e-12],
          [-2.85822610e-12, -2.85822610e-12]]],


        [[[-2.03267939e-12, -1.38918771e-12],
          [-2.03132709e-12, -1.38834737e-12]],

         [[-2.97604589e-12, -2.03390973e-12],
          [-2.97406599e-12, -2.03267939e-12]]]]])
Coordinates:
  * a        (a) int64 300 600
  * alpha    (alpha) float64 0.0 1.0
  * psi      (psi) float64 1.0 1.1
  * theta    (theta) float64 1.0 1.1
  * eta      (eta) float64 1.0 1.1

In [19]:
v_x_values = growth_stage.v_x_expectations(
    {'a' : [0, 500, 1000], 'alpha' : [0, 0.5, 1.0]},
    {
        'psi' : 4, 
        'theta' : 4, 
        'eta' : 4,
     # 'live' : [0, 1] 
    }, growth_v_y)

v_x_values

<xarray.DataArray (a: 3, alpha: 3)>
array([[-1.43219569e+03, -1.43219569e+03, -1.43219569e+03],
       [-4.11471265e-10, -8.56727914e-10, -1.76091488e-08],
       [-2.59114963e-11, -5.41234175e-11, -1.12927079e-09]])
Coordinates:
  * a        (a) int64 0 500 1000
  * alpha    (alpha) float64 0.0 0.5 1.0

In [20]:
growth_v_x = growth_stage.get_v_x(
    {'a' : [0, 250, 500, 750, 1000], 'alpha' : [0, 0.2, 0.4, 0.6, 0.8, 1.0]},
    {
        'psi' : 4, 
        'theta' : 4, 
        'eta' : 4,
     # 'live' : [0, 1] 
    }, growth_v_y)

growth_v_x({'a' : 300, 'alpha' : 0.25})

<xarray.DataArray ()>
array(-5.8995435e-09)
Coordinates:
    a        int64 300
    alpha    float64 0.25

## Infinite Horizon Solver

We chain together the stages and iterative solve each stage backwards, feeding $v_x$ into $v_y$, until convergence.

In [21]:
stages_data = [
    {
        'stage' : consumption_stage,
        'x_grid' : {'m' : range(0,500,50)}
    },
    {
        'stage' : allocation_stage,
        'x_grid' : {'a' : range(0,500,50)}
    },
    {
        'stage' : growth_stage,
        'x_grid' : {
            'a' : range(0,500,50),
            'alpha' : [0, 0.2, 0.4, 0.6, 0.8, 1.0]
        },
        'shock_approx_params' : {
            'psi' : 4, 
            'theta' : 4, 
            'eta' : 4,
        }
    }
]

In [25]:
def backwards_induction(stages_data, terminal_v_y):
    
    v_y = terminal_v_y
    
    v_y_list = []
    v_x_exp_list = []
    v_x_list = []
    
    for t in range(len(stages_data) - 1, -1, -1):
        print(t)
        
        v_y_list.insert(0,v_y)
        
        stage_data = stages_data[t]
        
        stage = stage_data['stage']
        x_grid = stage_data['x_grid']
        
        if 'shock_approx_params' in stage_data:
            shock_approx_params = stage_data['shock_approx_params']
        else:
            shock_approx_params = {}
            
        v_x_exp = stage.v_x_expectations(
            x_grid = x_grid,
            shock_approx_params = shock_approx_params,
            v_y = v_y
        )            

        v_x = stage.get_v_x(
            x_grid = x_grid,
            shock_approx_params = shock_approx_params,
            v_y = v_y
        )
        
        v_x_exp_list.insert(0, v_x_exp)
        v_x_list.insert(0, v_x)
        
        v_y = v_x
        
    return v_x_list, v_x_exp_list, v_y_list
        
v_x_list, v_x_exp_list, v_y_list = backwards_induction(stages_data, growth_v_y)
    


2
1
     fun: 3.6417768569663167e-06
   maxcv: 0.0
 message: 'NaN result encountered.'
    nfev: 3
  status: 5
 success: False
       x: array([0.])
     fun: 2.4285983808114114e-07
   maxcv: 0.0
 message: 'NaN result encountered.'
    nfev: 3
  status: 5
 success: False
       x: array([0.])
     fun: 4.9099381814449907e-08
   maxcv: 0.0
 message: 'NaN result encountered.'
    nfev: 3
  status: 5
 success: False
       x: array([0.])
     fun: 1.5722004321702387e-08
   maxcv: 0.0
 message: 'NaN result encountered.'
    nfev: 3
  status: 5
 success: False
       x: array([0.])
     fun: 6.486761747139437e-09
   maxcv: 0.0
 message: 'NaN result encountered.'
    nfev: 3
  status: 5
 success: False
       x: array([0.])
     fun: 3.143620622321209e-09
   maxcv: 0.0
 message: 'NaN result encountered.'
    nfev: 3
  status: 5
 success: False
       x: array([0.])
     fun: 1.7028348533299918e-09
   maxcv: 0.0
 message: 'NaN result encountered.'
    nfev: 3
  status: 5
 success: False
     

/home/sb/projects/HARK/HARK/utilities.py:141: RuntimeWarning: divide by zero encountered in double_scalars
  return c ** (1.0 - gam) / (1.0 - gam)
/home/sb/projects/HARK/HARK/utilities.py:141: RuntimeWarning: divide by zero encountered in double_scalars
  return c ** (1.0 - gam) / (1.0 - gam)
/home/sb/projects/HARK/HARK/utilities.py:141: RuntimeWarning: divide by zero encountered in double_scalars
  return c ** (1.0 - gam) / (1.0 - gam)
/home/sb/projects/HARK/HARK/utilities.py:141: RuntimeWarning: divide by zero encountered in double_scalars
  return c ** (1.0 - gam) / (1.0 - gam)
/home/sb/projects/HARK/HARK/utilities.py:141: RuntimeWarning: divide by zero encountered in double_scalars
  return c ** (1.0 - gam) / (1.0 - gam)
/home/sb/projects/HARK/HARK/utilities.py:141: RuntimeWarning: divide by zero encountered in double_scalars
  return c ** (1.0 - gam) / (1.0 - gam)
/home/sb/projects/HARK/HARK/utilities.py:141: RuntimeWarning: divide by zero encountered in double_scalars
  return c 

In [26]:
for x in v_x_exp_list:
    print(x)

<xarray.DataArray (m: 10)>
array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])
Coordinates:
  * m        (m) int64 0 50 100 150 200 250 300 350 400 450
<xarray.DataArray (a: 10)>
array([-1432.19568712,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan])
Coordinates:
  * a        (a) int64 0 50 100 150 200 250 300 350 400 450
<xarray.DataArray (a: 10, alpha: 6)>
array([[-1.43219569e+03, -1.43219569e+03, -1.43219569e+03,
        -1.43219569e+03, -1.43219569e+03, -1.43219569e+03],
       [-3.64177694e-06, -3.56959844e-06, -5.38583885e-06,
        -1.04489605e-05, -2.79130441e-05, -1.25508928e-04],
       [-2.42859838e-07, -2.38948273e-07, -3.64349566e-07,
        -7.18733748e-07, -1.97071841e-06, -9.24712599e-06],
       [-4.90993818e-08, -4.83809946e-08, -7.40812591e-08,
        -1.47136907e-07, -4.07979329e-07, -1.95225661e-06],
       [-1.57220043e-08, -1.55044005e-08, -2.37

In [34]:
v_x_list[1]({'a' : 120})

<xarray.DataArray ()>
array(nan)
Coordinates:
    a        int64 120

## 2022-10-7 notes


$x_2 = y_1 = T_1(x_1,k_1,a_1))$

$x_3 = T_2(x_2, k_2, \emptyset)$
